# Topic Modeling using BERTopic

In [97]:
!pip install bertopic

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Import libraries

In [119]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from transformers import pipeline
from wordcloud import WordCloud
import matplotlib.pyplot as plt 

from snorkel.labeling import LabelingFunction
import re
from snorkel.preprocess import preprocessor
from textblob import TextBlob

from tqdm import tqdm

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, PartOfSpeech, MaximalMarginalRelevance
from sklearn.metrics.pairwise import cosine_similarity

import spacy
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/martjebuss/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/martjebuss/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Load data

In [131]:
# Load df
df = pd.read_csv(
    "../data/chatgpt_short_clean_en.csv",
    encoding="utf-8",
)

df_long = pd.read_csv(
    "../data/chatgpt_long_clean_en.csv",
    encoding="utf-8",
)

## Remove NAs

In [132]:
df = df.dropna(subset=['content'])

In [101]:
len(df)

21352

## Delete reviews of one or two words

In [133]:
def has_multiple_words(text):
    words = text.split()
    return len(words) > 1

df = df[df['content'].apply(has_multiple_words)]


In [134]:
len(df)

21054

## BERTopic simple model

In [135]:
docs = list(df.content.values)
topic_model = BERTopic()

In [136]:
topics, probs = topic_model.fit_transform(docs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [137]:
topic_model.get_topic_info().head(15).set_index('Topic')[
   ['Count', 'Name', 'Representation']]

Count                                 Name  \
Topic                                               
-1      6595                    -1_and_it_this_to   
 0       390        0_chatgpt_mobile_official_app   
 1       362          1_voice_speech_text_feature   
 2       354      2_2021_september_update_updated   
 3       313                  3_ai_best_ever_seen   
 4       301   4_students_study_studies_education   
 5       287             5_log_login_sign_account   
 6       265           6_amazing_superb_love_this   
 7       236          7_history_chat_chats_scroll   
 8       207     8_answers_answer_questions_wrong   
 9       204        9_images_image_pictures_photo   
 10      200  10_students_homework_student_school   
 11      178                  11_good_app_chad_10   
 12      167        12_great_app_notice_favourite   
 13      153                 13_gpt_35_gpt35_free   

                                          Representation  
Topic                                                     
-1     [and, it, this, to, of, me, chatgpt, you, its,...  
 0     [chatgpt, mobile, official, app, finally, web,...  
 1     [voice, speech, text, feature, audio, recognit...  
 2     [2021, september, update, updated, data, 2023,...  
 3     [ai, best, ever, seen, app, this, apps, free, ...  
 4     [students, study, studies, education, student,...  
 5     [log, login, sign, account, cant, tried, passw...  
 6     [amazing, superb, love, this, app, awesome, su...  
 7     [history, chat, chats, scroll, bottom, previou...  
 8     [answers, answer, questions, wrong, give, ques...  
 9     [images, image, pictures, photo, picture, came...  
 10    [students, homework, student, school, assignme...  
 11    [good, app, chad, 10, mast, tbe, forever, once...  
 12    [great, app, notice, favourite, wonderful, pro...  
 13    [gpt, 35, gpt35, free, shortcut, are, release,...

In [138]:
topic_model.visualize_barchart(top_n_topics = 16, n_words = 10)

### Long format

In [109]:
docs = list(df_long.content.values)
topic_model = BERTopic()

In [110]:
topics, probs = topic_model.fit_transform(docs)

In [111]:
topic_model.get_topic_info().head(15).set_index('Topic')[
   ['Count', 'Name', 'Representation']]

Count                                Name  \
Topic                                              
-1     10143                     -1_and_me_it_of   
 0       686     0_2021_september_update_updated   
 1       563       1_chatgpt_mobile_official_app   
 2       435     2_voice_speech_text_recognition   
 3       383    3_students_studies_study_student   
 4       363    4_answers_answer_questions_gives   
 5       312   5_chats_chat_history_conversation   
 6       302  6_wrost_greeaaat_bellisimo_missing   
 7       289                 7_ai_best_seen_ever   
 8       252       8_chatgpt_help_invaluable_has   
 9       246     9_image_images_pictures_picture   
 10      223      10_great_app_wonderful_anyways   
 11      220          11_stars_star_five_ratings   
 12      203                12_gpt_gpt35_35_free   
 13      193               13_good_app_fore_shit   

                                          Representation  
Topic                                                     
-1        [and, me, it, of, in, to, this, can, for, its]  
 0     [2021, september, update, updated, data, 2023,...  
 1     [chatgpt, mobile, official, app, browser, webs...  
 2     [voice, speech, text, recognition, audio, feat...  
 3     [students, studies, study, student, exam, scho...  
 4     [answers, answer, questions, gives, question, ...  
 5     [chats, chat, history, conversation, previous,...  
 6     [wrost, greeaaat, bellisimo, missing, one, thi...  
 7     [ai, best, seen, ever, app, vary, ive, apps, o...  
 8     [chatgpt, help, invaluable, has, task, helping...  
 9     [image, images, pictures, picture, photo, came...  
 10    [great, app, wonderful, anyways, pocket, till,...  
 11    [stars, star, five, ratings, give, rating, rat...  
 12    [gpt, gpt35, 35, free, plus, 40, when, integra...  
 13    [good, app, fore, shit, chad, bull, mast, 10, ...

## Improving the topic model

#### Lemmatization

In [115]:
def lemma(text): 
    tokens = re.split(r"\s+",text)
    lemmatiser = WordNetLemmatizer()
    text = " ".join([lemmatiser.lemmatize(token) for token in tokens])
    return text 

In [139]:
def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

In [121]:
example_text = "lots of times the things that chatgpt says are wrong i often want to rely on chatgpt if i have questions i can't find an answer to with one single google search and it sometimes told me things that were simply wrong i can't trust the both also it doesn't know some products maybe it's time to update it's knowledge and a lot of times while chatting the system just collapses and i can't chat with the bot anymore for several minutes to hours even tho my internet connection is good"
print(example_text)
print(lemma(example_text))
print(lemmatizer(example_text))

lots of times the things that chatgpt says are wrong i often want to rely on chatgpt if i have questions i can't find an answer to with one single google search and it sometimes told me things that were simply wrong i can't trust the both also it doesn't know some products maybe it's time to update it's knowledge and a lot of times while chatting the system just collapses and i can't chat with the bot anymore for several minutes to hours even tho my internet connection is good
lot of time the thing that chatgpt say are wrong i often want to rely on chatgpt if i have question i can't find an answer to with one single google search and it sometimes told me thing that were simply wrong i can't trust the both also it doesn't know some product maybe it's time to update it's knowledge and a lot of time while chatting the system just collapse and i can't chat with the bot anymore for several minute to hour even tho my internet connection is good
lot of time the thing that chatgpt say be wrong

In [164]:
#copy = df.copy()
df = copy

In [122]:
df['content'] = df['content'].apply(lemma)

In [141]:
df['content'] = df['content'].apply(lemmatizer)

In [142]:
docs = list(df.content.values)
topic_model = BERTopic()

In [143]:
topics, probs = topic_model.fit_transform(docs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [144]:
topic_model.get_topic_info().head(15).set_index('Topic')[
   ['Count', 'Name', 'Representation']]

Count                            Name  \
Topic                                          
-1      6304            -1_it_and_chatgpt_be   
 0       619            0_ai_ever_world_tool   
 1       360     1_voice_speech_text_feature   
 2       338    2_chatgpt_official_app_thank   
 3       316    3_2021_update_september_2023   
 4       234              4_ai_ever_app_this   
 5       218       5_openai_team_thank_their   
 6       213         6_good_app_such_apphelp   
 7       209    7_answer_question_wrong_give   
 8       197    8_image_picture_photo_camera   
 9       168     9_great_app_chase_favourite   
 10      156  10_helpful_very_helpfully_wery   
 11      155           11_gpt_35_plain_gpt35   
 12      149  12_useful_very_verry_extremely   
 13      140  13_answer_question_give_second   

                                          Representation  
Topic                                                     
-1     [it, and, chatgpt, be, to, have, of, for, help...  
 0     [ai, ever, world, tool, intelligence, human, a...  
 1     [voice, speech, text, feature, audio, recognit...  
 2     [chatgpt, official, app, thank, website, mobil...  
 3     [2021, update, september, 2023, information, l...  
 4     [ai, ever, app, this, good, see, free, life, g...  
 5     [openai, team, thank, their, kudo, job, wait, ...  
 6     [good, app, such, apphelp, argument, oral, mas...  
 7     [answer, question, wrong, give, he, sometimes,...  
 8     [image, picture, photo, camera, video, add, sc...  
 9     [great, app, chase, favourite, notice, wonderf...  
 10    [helpful, very, helpfully, wery, indeed, much,...  
 11    [gpt, 35, plain, gpt35, free, allow, plus, pre...  
 12    [useful, very, verry, extremely, usefull, inte...  
 13    [answer, question, give, second, ask, every, a...

In [ ]:
topic_model.visualize_barchart(top_n_topics = 16, n_words = 10)

#### Tuning parameters

In [228]:
docs = list(df.content.values)
topic_model = BERTopic()

In [229]:
main_representation_model = KeyBERTInspired()
aspect_representation_model1 = PartOfSpeech("en_core_web_sm")
aspect_representation_model2 = [KeyBERTInspired(top_n_words=30), 
                                MaximalMarginalRelevance(diversity=.5)]

representation_model = {
   "Main": main_representation_model,
   "Aspect1":  aspect_representation_model1,
   "Aspect2":  aspect_representation_model2 
}

vectorizer_model = CountVectorizer(min_df=2, stop_words = 'english')
topic_model = BERTopic(nr_topics = 'auto', 
                      vectorizer_model = vectorizer_model,
                      representation_model = representation_model)


In [230]:
X = vectorizer_model.fit_transform(docs)
vocabulary = vectorizer_model.get_feature_names_out()
print(len(vocabulary))

5171


In [231]:
topics, ini_probs = topic_model.fit_transform(docs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [232]:
def get_topic_stats(topic_model, extra_cols = []):
    topics_info_df = topic_model.get_topic_info().sort_values('Count', ascending = False)
    topics_info_df['Share'] = 100.*topics_info_df['Count']/topics_info_df['Count'].sum()
    topics_info_df['CumulativeShare'] = 100.*topics_info_df['Count'].cumsum()/topics_info_df['Count'].sum()
    return topics_info_df[['Topic', 'Count', 'Share', 'CumulativeShare', 
                           'Name', 'Representation'] + extra_cols]

get_topic_stats(topic_model, ['Aspect1', 'Aspect2']).head(25)\
    .set_index('Topic')

Count      Share  CumulativeShare  \
Topic                                      
-1      7068  33.570818        33.570818   
 0      3344  15.882968        49.453786   
 1      1595   7.575758        57.029543   
 2       800   3.799753        60.829296   
 3       536   2.545835        63.375131   
 4       397   1.885627        65.260758   
 5       364   1.728888        66.989646   
 6       358   1.700389        68.690035   
 7       281   1.334663        70.024698   
 8       207   0.983186        71.007884   
 9       204   0.968937        71.976822   
 10      203   0.964187        72.941009   
 11      154   0.731452        73.672461   
 12      132   0.626959        74.299421   
 13      122   0.579462        74.878883   
 14      119   0.565213        75.444096   
 15      110   0.522466        75.966562   
 16      106   0.503467        76.470029   
 17      105   0.498718        76.968747   
 18       95   0.451221        77.419968   
 19       95   0.451221        77.871188   
 20       95   0.451221        78.322409   
 21       90   0.427472        78.749881   
 22       89   0.422723        79.172604   
 23       83   0.394224        79.566828   

                                                  Name  \
Topic                                                    
-1                     -1_app_apps_android_application   
 0                       0_2021_updates_latest_updated   
 1     1_chatgpt_chatgpts_conversational_conversations   
 2                   2_awesome_amazing_fantastic_great   
 3                  3_login_unable_reinstalled_account   
 4                    4_helpful_helpfully_helped_helps   
 5                              5_ai_apps_app_software   
 6                6_voicetotext_voice_microphone_audio   
 7                7_useful_usefulness_useless_purposes   
 8                 8_answers_answering_answered_answer   
 9                    9_telephone_phone_number_numbers   
 10                    10_images_photos_pictures_image   
 11                           11_gpt_gpt35_best_thanks   
 12                   12_prompt_prompts_editing_editor   
 13            13_bangladesh_bangladeshi_bengali_nepal   
 14                     14_android_androids_mobile_app   
 15              15_chat_chats_histories_conversations   
 16                      16_tool_tools_handy_fantastic   
 17                         17_openai_app_apps_android   
 18                18_ai_intelligence_best_intelligent   
 19             19_experience_experienced_amazing_best   
 20                 20_google_googles_googling_googled   
 21                       21_browser_web_slower_faster   
 22                22_chrome_chromebook_browser_unable   
 23    23_informative_useful_information_comprehensive   

                                          Representation  \
Topic                                                      
-1     [app, apps, android, application, mobile, chat...   
 0     [2021, updates, latest, updated, update, 2023,...   
 1     [chatgpt, chatgpts, conversational, conversati...   
 2     [awesome, amazing, fantastic, great, incredibl...   
 3     [login, unable, reinstalled, account, logged, ...   
 4     [helpful, helpfully, helped, helps, helping, u...   
 5     [ai, apps, app, software, applications, applic...   
 6     [voicetotext, voice, microphone, audio, speech...   
 7     [useful, usefulness, useless, purposes, purpos...   
 8     [answers, answering, answered, answer, gives, ...   
 9     [telephone, phone, number, numbers, cellphone,...   
 10    [images, photos, pictures, image, photo, camer...   
 11    [gpt, gpt35, best, thanks, quality, main, peak...   
 12    [prompt, prompts, editing, editor, modify, ent...   
 13    [bangladesh, bangladeshi, bengali, nepal, hind...   
 14    [android, androids, mobile, app, iphone, os, l...   
 15    [chat, chats, histories, conversations, conver...   
 16    [tool, tools, handy, fantastic, useful, marvel...   
 17    [openai, app, apps, android, application, th

In [260]:
topic_model.get_topic(9)

[('telephone', 0.55235445),
 ('phone', 0.4920091),
 ('number', 0.4854372),
 ('numbers', 0.43627506),
 ('cellphone', 0.41902268),
 ('phones', 0.41098958),
 ('calls', 0.38999927),
 ('address', 0.38979998),
 ('needs', 0.37257206),
 ('need', 0.36359888)]

In [261]:
topic_model.get_representative_docs(9)

['why do you need phone number not nessisary.',
 'why do you need my phone number',
 "couldn't even use because no phone number with"]

In [263]:
df_topic = pd.DataFrame({"topic": topics, "document": docs})
df_topic

topic                                           document
0          1  chatgpt on android is a solid app with seamles...
1          6  i've been using chatgpt for a while but i've j...
2         -1  the chatgpt android app has completely blown m...
3          0  no subscription free and accurate unbiased ans...
4          1  i use this app for learning languages which ch...
...      ...                                                ...
21049     66                                       first review
21050     47                                   first downloaded
21051      0                                        usually app
21052     56                                      first comment
21053     -1                                          hum first

[21054 rows x 2 columns]

### Visualization

In [154]:
topic_model.visualize_barchart(top_n_topics = 21, n_words = 10)

In [156]:
topic_model.visualize_topics()

### Embedding model

## Add topics to df

## Search topics

Note: You can only use this method if an embedding model was supplied to BERTopic using embedding_model.

In [226]:
similar_topics, similarity = topic_model.find_topics("voice", top_n=5)
topic_model.get_topic(similar_topics[0])

[('voicetotext', 0.7687919),
 ('voice', 0.64830846),
 ('speechtotext', 0.62790406),
 ('microphone', 0.5031773),
 ('audio', 0.48563433),
 ('speech', 0.46069753),
 ('transcription', 0.45139158),
 ('speaker', 0.41808367),
 ('text', 0.41523013),
 ('speak', 0.40731138)]

## Topics per Class 

### App Rating (positive, neutral, negative)

In [167]:
topics_per_class = topic_model.topics_per_class(docs, classes=df.score_cat)
fig = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=15, 
                                      normalize_frequency = True)

fig.write_html('basic.html')
fig.show()

### App Version 

In [189]:
df_version = df.dropna(subset=['appVersion'])

In [199]:
df_version['content'] = df_version['content'].apply(lemmatizer)

In [200]:
docs = list(df_version.content.values)
topic_model = BERTopic()

In [201]:
main_representation_model = KeyBERTInspired()
aspect_representation_model1 = PartOfSpeech("en_core_web_sm")
aspect_representation_model2 = [KeyBERTInspired(top_n_words=30), 
                                MaximalMarginalRelevance(diversity=.5)]

representation_model = {
   "Main": main_representation_model,
   "Aspect1":  aspect_representation_model1,
   "Aspect2":  aspect_representation_model2 
}

vectorizer_model = CountVectorizer(min_df=2, stop_words = 'english')
topic_model = BERTopic(nr_topics = 'auto', 
                      vectorizer_model = vectorizer_model,
                      representation_model = representation_model)

topics, ini_probs = topic_model.fit_transform(docs)
def get_topic_stats(topic_model, extra_cols = []):
    topics_info_df = topic_model.get_topic_info().sort_values('Count', ascending = False)
    topics_info_df['Share'] = 100.*topics_info_df['Count']/topics_info_df['Count'].sum()
    topics_info_df['CumulativeShare'] = 100.*topics_info_df['Count'].cumsum()/topics_info_df['Count'].sum()
    return topics_info_df[['Topic', 'Count', 'Share', 'CumulativeShare', 
                           'Name', 'Representation'] + extra_cols]

get_topic_stats(topic_model, ['Aspect1', 'Aspect2']).head(25)\
    .set_index('Topic')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Count      Share  CumulativeShare  \
Topic                                      
 0      6644  37.185873        37.185873   
-1      5975  33.441540        70.627414   
 1       332   1.858174        72.485588   
 2       246   1.376840        73.862428   
 3       202   1.130576        74.993004   
 4       180   1.007444        76.000448   
 5       160   0.895506        76.895953   
 6       148   0.828343        77.724296   
 7       146   0.817149        78.541445   
 8       119   0.666032        79.207477   
 9        98   0.548497        79.755975   
 10       87   0.486931        80.242906   
 11       82   0.458947        80.701853   
 12       81   0.453350        81.155202   
 13       79   0.442156        81.597358   
 14       76   0.425365        82.022723   
 16       73   0.408574        82.431298   
 15       73   0.408574        82.839872   
 17       72   0.402978        83.242850   
 18       72   0.402978        83.645828   
 19       70   0.391784        84.037611   
 20       62   0.347008        84.384620   
 21       62   0.347008        84.731628   
 22       60   0.335815        85.067443   
 23       59   0.330218        85.397660   

                                              Name  \
Topic                                                
 0                      0_chatgpt_app_chat_android   
-1                     -1_chatgpt_app_android_chat   
 1                   1_awesome_tomorrow_hype_wanna   
 2     2_voicetotext_voice_speechtotext_microphone   
 3                     3_openai_thank_thanks_happy   
 4                 4_answer_response_reply_inquiry   
 5                    5_image_photo_camera_picture   
 6               6_number_phone_cellphone_register   
 7                          7_gpt_gpt35_masse_good   
 8                              8_good_great_best_   
 9                    9_browser_web_webpage_chrome   
 10           10_information_info_informative_data   
 11                   11_use_useful_easy_practical   
 12                12_prompt_editing_modify_submit   
 13          13_microdot_login_account_loginsignup   
 14        14_equation_latex_formulae_mathematical   
 16               16_experience_amazing_thank_glad   
 15         15_writing_writer_storytelle_narrative   
 17             17_performance_slow_excellent_fast   
 18                  18_response_fast_quickly_slow   
 19     19_homework_project_assignment_description   
 20    20_interpreter_code_programming_pluginscode   
 21                        21_help_aid_lot_improve   
 22                    22_datum_2023_2021_outdated   
 23               23_history_sidebar_recent_search   

                                          Representation  \
Topic                                                      
 0     [chatgpt, app, chat, android, openai, conversa...   
-1     [chatgpt, app, android, chat, application, mob...   
 1     [awesome, tomorrow, hype, wanna, super, specta...   
 2     [voicetotext, voice, speechtotext, microphone,...   
 3     [openai, thank, thanks, happy, forward, respec...   
 4     [answer, response, reply, inquiry, ask, questi...   
 5     [image, photo, camera, picture, upload, graphi...   
 6     [number, phone, cellphone, register, require, ...   
 7     [gpt, gpt35, masse, good, case, nt, support, m...   
 8                     [good, great, best, , , , , , , ]   
 9     [browser, web, webpage, chrome, browsing, vers...   
 10    [information, info, informative, data, informe...   
 11    [use, useful, easy, practical, simple, conveni...   
 12    [prompt, editing, modify, submit, customizatio...   
 13    [microdot, login, account, loginsignup, passwo...   
 14    [equation, latex, formulae, mathematical, form...   
 16    [experience, amazing, thank, glad, good, great...   
 15    [writing, writer, storytelle, narrative, write...   
 17    [performance, slow, excellent, fast, efficient...   
 18    [response, fast, quickly, slow, respond, speed...   
 19    [homework, project, assignment, 

In [202]:
topics_per_class = topic_model.topics_per_class(docs, classes=df_version.appVersion)

In [203]:
fig = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=15, 
                                      normalize_frequency = True)

fig.write_html('basic.html')
fig.show()

## Timeline

In [184]:
docs = list(df.content.values)
topic_model = BERTopic()

In [185]:
topics_per_class = topic_model.topics_per_class(docs, classes=df.at_m)
fig = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=15, 
                                      normalize_frequency = True)

fig.write_html('basic.html')
fig.show()

ValueError: Expected 2D array, got scalar array instead:
array=nan.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [165]:
df.head()

Unnamed: 0                              reviewId        userName  \
0           0  36b7f28e-151d-4b98-8a13-41bd017e0d25       Lin Cheng   
1           1  2bc253b6-c804-47e9-b6f2-3a21027efab4            Alim   
2           2  5f084727-ab85-40b3-bd42-a7a49502fc1f      Theo Healy   
3           3  5df90de5-b8e2-4dc2-b6ff-520aa3a25eae  Elliot Limberg   
4           4  bb66c666-865d-4a31-b27f-4933df3ff829   Phoebe Moraes   

                                             content  score  thumbsUpCount  \
0  chatgpt on android is a solid app with seamles...      4              5   
1  i've been using chatgpt for a while but i've j...      5            139   
2  the chatgpt android app has completely blown m...      4            247   
3  no subscription free and accurate unbiased ans...      5            272   
4  i use this app for learning languages which ch...      4            126   

                    at replyContent repliedAt  appVersion    at_ymd  at_q  \
0  2023-10-19 19:26:19          NaN       NaN  1.2023.284  10/19/23     4   
1  2023-09-29 20:24:38          NaN       NaN  1.2023.263  09/29/23     3   
2  2023-07-28 10:29:10          NaN       NaN    1.0.0023  07/28/23     3   
3  2023-07-30 19:38:37          NaN       NaN    1.0.0023  07/30/23     3   
4  2023-08-09 18:23:33          NaN       NaN    1.0.0030  08/09/23     3   

     at_ym       at_m      at_wd score_cat detected_language  
0  2023-10    October   Thursday   neutral                en  
1  2023-09  September     Friday  positive                en  
2  2023-07       July     Friday   neutral                en  
3  2023-07       July     Sunday  positive                en  
4  2023-08     August  Wednesday   neutral                en

Summary: 
- Bugs with login, other errors  
- Educational purpose, studying 
- Helping, Assistance 
- Phone number for registration needed, verification
- Feature request on uploading photos, gettung responses with images 
- Translation problems for hindi speakers 
- future-oriented topic

## Dealing with Outliers

-1 represents outliers: 6841! 

In [67]:
percentage_outliers = (6841 / 22371)*100 
percentage_outliers

30.579768450225735

Four different strategies to deal with the outliers:
1. based on topic-document probabilities,
2. based on topic distributions,
3. based on c-TF-IFD representations,
4. based on document and topic embeddings.

## Reducing the number of topics

In [158]:
topic_model.visualize_heatmap(n_clusters = 20)

In [159]:
distance_matrix = cosine_similarity(np.array(topic_model.topic_embeddings_))
dist_df = pd.DataFrame(distance_matrix, columns=topic_model.topic_labels_.values(), 
                       index=topic_model.topic_labels_.values())

tmp = []
for rec in dist_df.reset_index().to_dict('records'):
    t1 = rec['index']
    for t2 in rec:
        if t2 == 'index': 
            continue
        tmp.append(
            {
                'topic1': t1, 
                'topic2': t2, 
                'distance': rec[t2]
            }
        )

pair_dist_df = pd.DataFrame(tmp)

pair_dist_df = pair_dist_df[(pair_dist_df.topic1.map(
      lambda x: not x.startswith('-1'))) & 
            (pair_dist_df.topic2.map(lambda x: not x.startswith('-1')))]
pair_dist_df = pair_dist_df[pair_dist_df.topic1 < pair_dist_df.topic2]
pair_dist_df.sort_values('distance', ascending = False).head(20)

topic1  \
40635                      200_android_app_native_phone   
9373                45_review_reviewer_follow_complaint   
28134                        138_use_easy_simple_easily   
377                      0_app_application_use_download   
346                      0_app_application_use_download   
426          1_chatgpt_conversation_conversational_chat   
22378            109_grammar_grammatical_editing_speech   
4889                     23_app_2021_2023_currentupdate   
27043                 132_chatgpt_chatgpts_login_unable   
3677                      17_app_webview_mobile_version   
26666                          131_chat_app_chatty_talk   
32734         161_app_knowledge_educational_development   
4678                       22_openai_app_android_mobile   
3506   16_application_beneficial_recommendation_benefit   
28514                140_openai_ai_architecture_improve   
243                      0_app_application_use_download   
10918                        53_app_essay_writing_write   
10373                      50_helpful_tool_useful_handy   
29373                          144_ca_login_register_nt   
304                      0_app_application_use_download   

                                           topic2  distance  
40635                32_android_mobile_finally_os  0.887677  
9373              80_review_reviewer_rating_thank  0.887035  
28134               55_useful_use_practical_handy  0.885001  
377              174_app_experience_use_wonderful  0.878171  
346              143_app_interesting_neat_curious  0.869638  
426                21_chatgpt_openai_chat_android  0.855959  
22378    157_app_language_application_translation  0.854126  
4889            40_app_improvement_improve_appian  0.853135  
27043               176_chatgpt_unable_error_wifi  0.851943  
3677            40_app_improvement_improve_appian  0.850428  
26666  1_chatgpt_conversation_conversational_chat  0.844870  
32734             9_app_student_studentsit_useful  0.844811  
4678                 31_openai_thank_thanks_happy  0.843065  
3506           71_application_useful_software_app  0.841202  
28514                31_openai_thank_thanks_happy  0.838115  
243             40_app_improvement_improve_appian  0.834385  
10918             9_app_student_studentsit_useful  0.833853  
10373          70_tool_handy_fantastic_incredible  0.831285  
29373                     82_login_ca_log_connect  0.830136  
304          101_playstore_app_mobile_application  0.827567

### Merging

Intentionally left blank as we should take a deeper look into the clusters to see what can be merged

In [ ]:
#topic_model.merge_topics(docs, [[26, 74], [43, 68, 62], [16, 50, 91]])
#df['merged_topic'] = topic_model.topics_

## Topic Distributions

In [160]:
topic_distr, topic_token_distr = topic_model.approximate_distribution(
      docs, window = 4, calculate_tokens=True)

In [161]:
topic_model.visualize_distribution(topic_distr[1])

In [162]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(docs, calculate_tokens=True)

# Visualize the token-level distributions
df = topic_model.visualize_approximate_distribution(docs[1], topic_token_distr[1])
df

TO-DO: 

- append topic names to df 
- merge similar topics
